# Master SQL Fundamentals Effortlessly as a Pandas User
## There was a time it was the other way around
![](images/unsplash.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@itssammoqadam?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Sam Moqadam</a>
        on 
        <a href='https://unsplash.com/?utm_source=unsplash&utm_medium=referral&utm_content=creditCopyText'>Unsplash</a>
    </strong>
</figcaption>

### Motivation

When Pandas package gained public exposure in 2009, SQL had been dominating the data world since 1974. Pandas came with an attractive set of features such as in-built visualization, flexible data handling and became an ultimate data exploration tool. As it started gaining mass popularity, many courses and resources emerged teaching Pandas and comparing it to SQL.

Flash-forward to 2021, people are now getting introduced to the Pandas package *first* rather than the universal data language - SQL. Even though SQL is as popular as ever, the flexibility and multi-functionality of Pandas is making it the first choice for beginner data scientists.

Then, why do you need SQL if you know Pandas?

Even though Pandas may seem a better choice, SQL still plays a crucial role in day-to-day job of a data scientist. In fact, SQL is the second most in-demand and the third most growing programming language for data science (see [here](https://towardsdatascience.com/the-most-in-demand-skills-for-data-scientists-in-2021-4b2a808f4005)). So, it is a must to add SQL to your CV if you want to get a job in the field. And knowing Pandas, learning SQL should be a breeze, as you will see in this article.

### Connecting to a Database

Setting up an SQL workspace and connecting to a database can be a real pain in the neck. First, you need to install your favorite SQL flavor (PostgreSQL, MySQL, etc.) and download an SQL IDE too. Doing all those here would deviate us from the purpose of the article, so we will use a shortcut. 

Specifically, we will directly run SQL queries in a Jupyter Notebook without any additional steps. All we need to do is install the `ipython-sql` package using pip:

```python
pip install ipython-sql
```

After the installation is done, start a new Jupyter session and run this command in the notebook:

```python
%load_ext sql
```
and we are all set!

To illustrate how basic SQL statements work, we will be using the Chinook database which can be downloaded [here](https://www.sqlitetutorial.net/sqlite-sample-database/). The database has 11 tables, each of which has its own name. 

To retrieve the data stored in this database's tables, run this command:

In [34]:
%sql sqlite:///data/chinook.db

The statement starts with `%sql` in-line magic command that tells the notebook interpreter we will be running SQL commands. It is followed by the path that our downloaded Chinook database is in. The valid paths should always start with `sqlite:///` prefix for SQLite databases. Above, we are connecting to the database that is stored in the 'data' folder of the current directory. If you want to pass an absolute path, the prefix should be with 4 forward slashes - `sqlite:////`

If you wish to connect to a different database flavor, you can refer to this [excellent article](https://towardsdatascience.com/heres-how-to-run-sql-in-jupyter-notebooks-f26eb90f3259).

### Taking a First Look at the Tables

The first thing we always do in Pandas is use the `.head()` function. Let's learn how to do that in SQL.

The first and most used command in SQL is `SELECT`. This keyword can behave in two ways: 
- as a print function (rarely used, will learn about later)
- as a keyword to display tables, columns

Let's look at all the columns of the customers table in the Chinook database:

In [31]:
query = "SELECT * FROM customers LIMIT 5"

pd.read_sql_query(query, engine)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


Pandas has a `read_sql_query` function which takes an SQL query and a connection object as arguments. Let's explain the query string:

In SQL, `SELECT *` means displaying all columns from a table specified after the `FROM` keyword. So, the above query 